In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split
import os
import json
import random

In [ ]:
import torch
torch.__version__

In [14]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("m42-health/Llama3-Med42-8B")
model = AutoModelForCausalLM.from_pretrained("m42-health/Llama3-Med42-8B").to('cuda')

KeyError: 'llama'

In [9]:
prompt = '''
**conversation**
Doctor: Hello, I am Dr. Smith. Can you tell me what brings you to the hospital today?
Patient: Yes, I have been feeling very weak and sick for the past two weeks. I have a persistent fever and dry cough.
Doctor: I see. And how is your breathing?
Patient: It's been shallow and rapid, especially when I am at rest. And I get severely breathless even with minor physical activities.
Doctor: Okay. I understand. You were given physical therapy, right?
Patient: Yes, they focused on educating me about dyspnea-relieving positions and the importance of regular mobilization and deep-breathing exercises.
Doctor: That's good. And how did it go?
Patient: It became evident that my anxiety from fear of dying and worries about my future was making my dyspnea worse. I was so dyspneic, anxious, and weak that I could barely walk to the toilet.
Doctor: I see. But your physical therapist helped you with that, right?
Patient: Yes, they listened to me, explained why I was experiencing breathlessness, and tested suitable positions to relieve my dyspnea. I felt better after the education and relaxing breathing exercises.
Doctor: That's great to hear. Can you tell me more about the improvement?
Patient: On day 2, my respiratory rate reduced from 30 breaths/min to 22 breaths/min and my oxygen saturation increased from 92% to 96% on 4 L/min oxygen after some deep-breathing exercises.
Doctor: That's impressive. And how did it go after that?
Patient: My dyspnea and anxiety started to alleviate and I regained my self-confidence. The therapy was shifted to walking and strength training, and I was able to walk 350 m without a walking aid or supplemental oxygen before my discharge home.
Doctor: That's fantastic. You have made a great progress. I am glad to hear that.
**request**
Make medical letter with whole sentencse.
Clinic summary, Treatment Plan and Management, Progress and Outcomes, Follow-Up and Monitoring is required
answer :
**Clinical Summary:
Mr. X was admitted with a two-week history of fever, dry cough, and severe weakness. He had shallow, rapid breathing, especially at rest, and struggled with breathlessness even with minor activity. His anxiety worsened his breathing difficulties.

**Treatment Plan and Management:
Physical therapy focused on dyspnea-relieving positions, breathing exercises, and anxiety reduction techniques. Mr. X learned ways to manage his breathlessness and anxiety. Therapy also included education on daily mobilization.

**Progress and Outcomes:
By day 2, his respiratory rate dropped from 30 to 22 breaths per minute, and his oxygen levels improved from 92% to 96% with 4 L/min oxygen. His dyspnea and anxiety decreased, and his self-confidence improved. He could walk 350 meters without aid or supplemental oxygen at discharge.

**Follow-Up and Monitoring:
Continued breathing exercises and light physical activity at home were recommended. Regular follow-up visits will help monitor his progress. Support for anxiety management was also advised to aid recovery.e patient is advised to monitor their symptoms and keep in touch with their healthcare provider for follow-up appointments. Regular check-ins will be necessary to ensure continued progress and address any potential issues as they recover at home.

**conversation**
Doctor: Hi, Mr. X, I'm Dr. Y. How are you feeling today?
Patient: Not too good, doctor. I've been feeling really sick lately.
Doctor: I understand. Can you tell me what symptoms you're experiencing?
Patient: Yes, I've been having a fever, a dry cough, and dyspnea.
Doctor: I see. You were hospitalized due to moderate ARDS from COVID-19, is that correct?
Patient: Yes, that's correct.
Doctor: During your physical therapy, we encountered some difficulties. Can you tell me more about that?
Patient: Yes, I had trouble with position changes and deep breathing. Every time I tried to change my position or take a deep breath, I would start coughing and it would make me really short of breath.
Doctor: I understand. To avoid rapid deterioration and respiratory failure, we instructed you to change positions very slowly and step-by-step, right?
Patient: Yes, that's right. It took about 30 minutes to change to the prone position.
Doctor: And I see that this approach increased your oxygen saturation, for example, on day 5 with 6 L/min of oxygen from 93% to 97%.
Patient: Yes, that's correct.
Doctor: Good. We also had to adapt your breathing exercises to avoid prolonged coughing and oxygen desaturation. Can you tell me more about that?
Patient: Yes, I was instructed to stop every deep breath before coughing and to hold my breath for better air distribution.
Doctor: I see that you performed the breathing exercises well and managed to increase your oxygen saturation.
Patient: Yes, I did my best.
Doctor: You also had difficulty maintaining sufficient oxygen saturation during physical activity, is that correct?
Patient: Yes, I did. But with close monitoring and frequent breaks, I was able to perform low-level strength and walking exercises without any significant deoxygenation.
Doctor: I see that your exercise progression was low on days 1 to 5, but then increased daily until your hospital discharge to a rehabilitation clinic on day 10.
Patient: Yes, that's correct.
Doctor: Great. I'd like to keep monitoring your progress and see how you're doing. Can you keep me updated on any changes in your symptoms?
Patient: Yes, of course, doctor.
Doctor: Alright, let's keep in touch. If you have any questions or concerns, don't hesitate to reach out to me.
Patient: Thank you, doctor.
**request**
Make medical letter with whole sentencse.
Clinic summary, Treatment Plan and Management, Progress and Outcomes, Follow-Up and Monitoring is required
answer :
**Clinic Summary:
Mr. X was hospitalized for moderate ARDS due to COVID-19, experiencing fever, dry cough, and breathlessness. Physical therapy was challenging due to dyspnea, particularly during position changes and deep-breathing exercises.

**Treatment Plan and Management:
A gradual approach to position changes was used to prevent respiratory distress. Adjusted breathing exercises focused on controlled breaths to improve oxygen levels. Low-intensity walking and strength exercises were monitored closely to avoid deoxygenation.

**Progress and Outcomes:
By day 5, Mr. X’s oxygen saturation improved from 93% to 97% with 6 L/min of oxygen. He gradually increased activity tolerance and was discharged to rehabilitation on day 10.

**Follow-Up and Monitoring:
Regular follow-up and symptom monitoring are recommended, with continued gradual activity progression in rehabilitation.



**conversation**
Doctor: Good morning, how are you feeling today?
Patient: Not so good, doctor. I have been feeling short of breath and my chest feels tight.
Doctor: I see. Can you tell me more about what happened?
Patient: Well, I tested positive for COVID-19 and was admitted to the ICU a week ago because of my oxygen levels.
Doctor: Yes, I have your medical records here. It says you were admitted to the ICU due to oxygen desaturation of 70%. Is that correct?
Patient: Yes, that's right.
Doctor: And you were also experiencing worsening tachypnea and dyspnea, correct?
Patient: Yes, I was having a hard time breathing and my breathing was getting faster.
Doctor: I understand. Physical therapy started immediately after your ICU admission, is that correct?
Patient: Yes, that's correct.
Doctor: When you were admitted to the ICU, you were a highly dyspneic patient with a high breathing frequency, correct?
Patient: Yes, I was.
Doctor: I see. With the help of physical therapy guidance, you managed to achieve a 135° prone position and to perform deep-breathing exercises, which resulted in an increase in your oxygen saturation from 88% to 96%, correct?
Patient: Yes, that's correct.
Doctor: And after that, intensive physical therapy and positioning were continued along with 6 to 12 L/min of oxygen therapy over the next days, and intubation was avoided, correct?
Patient: Yes, that's right.
Doctor: I understand that the major challenges in achieving a prone position were your reduced respiratory capacity and the high risk of exacerbating your symptoms, correct?
Patient: Yes, that's correct.
Doctor: But standard ICU monitoring allowed us to safely implement physical therapy at an individually adapted pace, which allowed sufficient time for convalescence, correct?
Patient: Yes, that's right.
Doctor: After 3 days with this regime, you were transferred to the normal ward, where physical therapists carried on your rehabilitation with walking and strength training, correct?
Patient: Yes, that's correct.
Doctor: And despite your severe instability, 9 days after ICU admission, you were able to leave the hospital as a pedestrian, correct?
Patient: Yes, that's correct.
Doctor: That's great to hear. I'm glad you were able to recover from your illness.
**request**
Make medical letter with whole sentencees.
Clinic summary, Treatment Plan and Management, Progress and Outcomes, Follow-Up and Monitoring is required.
Please generate a summary based only on the following conversation. Do not include any information that is not clearly verified in the dialogue.
answer :

'''

In [8]:
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(
    inputs["input_ids"],
    max_new_tokens=400,
    do_sample=True,
    top_k=10,
    temperature=.3,
)
# 생성된 문장 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

NameError: name 'model' is not defined